In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
amp = 10.0
xref = 1.0
g1 = 0.57
g2 = -0.23
x0 = 1e-3

def func(xx):
    lo = (xx < xref)
    hi = ~lo
    yy = np.zeros_like(xx)
    yy[lo] = amp * np.power(xx[lo]/xref, g1)
    yy[hi] = amp * np.power(xx[hi]/xref, g2)
    return yy

def integral(xx):
    lo = (xx < xref)
    hi = ~lo
    
    yy = np.zeros_like(xx)
    yy[lo] = (amp / (g1+1)) * ((xx[lo] * (xx[lo]/xref)**g1) - (x0 * (x0/xref)**g1))    
    
    floor = (amp / (g1+1)) * (xref - x0 * (x0/xref)**g1)
    yy[hi] = floor + (amp / (g2+1)) * ((xx[hi] * (xx[hi]/xref)**g2) - xref)
    return yy    

xx = np.logspace(np.log10(x0), 3, 1000)
yy = func(xx)
zz = integral(xx)
print(f"{zz[-1]=:.4e}")

fig, ax = plt.subplots(figsize=[15, 10])
ax.set(xscale='log', yscale='log')
ax.grid(True)
tw = ax.twinx()
tw.set(yscale='log')
cc, = ax.plot(xx, yy)
tw.plot(xx, zz, color=cc.get_color(), ls='--')


NUM = 7
aa = np.logspace(np.log10(x0), 3, NUM)
bb = func(aa)
da = np.diff(aa)
b1l = np.cumsum(bb[:-1]*da)
b1r = np.cumsum(bb[1:]*da)
b1 = 0.5 * (b1l + b1r)
tw.plot([], [])
cc, = tw.plot(aa[1:], b1)

b2 = trapz_loglog(bb, aa)
print(aa.shape, bb.shape, b2.shape)
tw.plot(aa[1:], b2)

plt.show()
    

In [ ]:
ztrue = 10.0 ** np.interp(np.log10(aa[1:]), np.log10(xx), np.log10(zz))
d1 = (b1 - ztrue) / ztrue
d2 = (b2 - ztrue) / ztrue

fig, ax = plt.subplots(figsize=[10, 6])
ax.set(xscale='log', yscale='log')

for dd in [d1, d2]:
    plt.plot(aa[1:], np.fabs(dd))
    
plt.show()